# JAWS 
## Analizando los ataques de tiburones

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
sa_initial = pd.read_csv('Inputs/attacks.csv', engine='python')

# Datos iniciales

In [3]:
sa_initial

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
t_rows = sa_initial.shape[0]
print(t_rows)

25723


In [5]:
sa_initial.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

## Validez de los datos

In [6]:
def valid_columns(x):
    y=x.isnull().sum().sort_values(ascending=False)
    return y
valid_items = valid_columns(sa_initial)
print (valid_items)

Unnamed: 22               25722
Unnamed: 23               25721
Time                      22775
Species                   22259
Age                       22252
Sex                       19986
Activity                  19965
Location                  19961
Fatal (Y/N)               19960
Area                      19876
Name                      19631
Country                   19471
Injury                    19449
Investigator or Source    19438
Type                      19425
Year                      19423
href formula              19422
pdf                       19421
href                      19421
Case Number.1             19421
Case Number.2             19421
Date                      19421
original order            19414
Case Number               17021
dtype: int64


**Ratios de filas llenas vs número de filas (por columna):**

In [7]:
valid_ratio = pd.Series([(t_rows-x)/t_rows for x in valid_items])
print(valid_ratio)

0     0.000039
1     0.000078
2     0.114606
3     0.134665
4     0.134938
5     0.223030
6     0.223846
7     0.224002
8     0.224041
9     0.227306
10    0.236831
11    0.243051
12    0.243906
13    0.244334
14    0.244839
15    0.244917
16    0.244956
17    0.244995
18    0.244995
19    0.244995
20    0.244995
21    0.244995
22    0.245267
23    0.338296
dtype: float64


# Limpieza de datos

## Eliminando las columnas vacías 

In [8]:
sa_empty_rows=sa_initial.drop(columns=['Unnamed: 23', 'Unnamed: 22'])
sa_empty_rows

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Eliminando las filas vacías

In [9]:
columns = list(sa_empty_rows.columns)
intrested_columns=['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ']
sa_clean = sa_empty_rows
#como lo hago sin crear una copia
sa_clean.dropna(how='all', subset=intrested_columns, inplace=True)
sa_clean

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8�N, 79�W",NaN,Jules Patterson,M,...,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0


## Renombrando columnas

In [10]:
sa_clean.rename(columns={'Sex ':'Sex',
                          'Species ':'Species',
                          'Fatal (Y/N)':'Fatal'}, 
                 inplace=True)

In [11]:
sa_clean

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal,Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8�N, 79�W",NaN,Jules Patterson,M,...,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0


# Analizando los Datos

## Validez de los datos: 

Según el DataFrame limpio, muchos de los atques registrados no contienen la información de la hora del ataque, la especie de tiburón y la edad de la victima. 

Mis primeras hipótesis se basarán en aquellos aspectos que contienen una muestra más grande. 

In [12]:
valid_columns(sa_clean)

Time                      3354
Species                   2838
Age                       2831
Sex                        565
Activity                   544
Location                   540
Fatal                      539
Area                       455
Name                       210
Country                     50
Injury                      28
Investigator or Source      17
Type                         4
Year                         2
Case Number                  1
href formula                 1
Case Number.2                0
pdf                          0
href                         0
Case Number.1                0
Date                         0
original order               0
dtype: int64

## Ataques por año 

Eliminando filas que corresponden al año 0.0 
Creando una nueva base de datos con los ataques que tienen un año registrado

In [13]:
%matplotlib inline
x=sa_clean.Year.value_counts(ascending=False).head(60)
x=x.sort_index(ascending=True)
x

0.0       125
1929.0     38
1942.0     41
1950.0     43
1954.0     42
1955.0     43
1956.0     51
1957.0     41
1958.0     54
1959.0     93
1960.0     93
1961.0     78
1962.0     86
1963.0     61
1964.0     66
1965.0     51
1966.0     58
1967.0     48
1968.0     47
1970.0     42
1974.0     38
1975.0     49
1976.0     39
1981.0     49
1982.0     40
1983.0     50
1984.0     41
1985.0     37
1986.0     39
1988.0     55
1989.0     53
1990.0     38
1991.0     38
1992.0     56
1993.0     56
1994.0     56
1995.0     76
1996.0     61
1997.0     57
1998.0     65
1999.0     66
2000.0     97
2001.0     92
2002.0     88
2003.0     92
2004.0     92
2005.0    103
2006.0    103
2007.0    112
2008.0    122
2009.0    120
2010.0    101
2011.0    128
2012.0    117
2013.0    122
2014.0    127
2015.0    143
2016.0    130
2017.0    136
2018.0     53
Name: Year, dtype: int64

In [14]:
sa_clean.loc[sa_clean['Year'] == 0,'Year'] = np.nan

In [15]:
x=sa_clean.Year.value_counts(ascending=False).head(60)
x=x.sort_index()
x

1929.0     38
1942.0     41
1950.0     43
1953.0     36
1954.0     42
1955.0     43
1956.0     51
1957.0     41
1958.0     54
1959.0     93
1960.0     93
1961.0     78
1962.0     86
1963.0     61
1964.0     66
1965.0     51
1966.0     58
1967.0     48
1968.0     47
1970.0     42
1974.0     38
1975.0     49
1976.0     39
1981.0     49
1982.0     40
1983.0     50
1984.0     41
1985.0     37
1986.0     39
1988.0     55
1989.0     53
1990.0     38
1991.0     38
1992.0     56
1993.0     56
1994.0     56
1995.0     76
1996.0     61
1997.0     57
1998.0     65
1999.0     66
2000.0     97
2001.0     92
2002.0     88
2003.0     92
2004.0     92
2005.0    103
2006.0    103
2007.0    112
2008.0    122
2009.0    120
2010.0    101
2011.0    128
2012.0    117
2013.0    122
2014.0    127
2015.0    143
2016.0    130
2017.0    136
2018.0     53
Name: Year, dtype: int64

## Tipos de ataques

Hay una confusión en lo que se refiere a tipos de ataques, algunos están clasificados como provocados/no provocados, mientras otros registran la actividad.

Por eso cambiaré aquellos que contienen la actividad por NaN.

In [16]:
sa_clean.Type.value_counts().sum()

6298

In [17]:
sa_clean.Type.value_counts().head(50)

Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: Type, dtype: int64

(Intentando pasar de tipos a actividades mediante una función) ... No conseguido

`def add_activity(x):
    if x=='Invalid'|'Sea Disaster'|'Boating'|'Boat'|'Questionable'|'Boatomg':`

In [18]:
activities = ['Invalid','Sea Disaster','Boating','Boat','Questionable','Boatomg','Type']

for x in activities:
    sa_clean.loc[sa_clean['Type'] == x,'Type'] = np.nan

In [19]:
sa_clean.Type.value_counts().head(50)

Unprovoked    4595
Provoked       574
Name: Type, dtype: int64

## Actividad
La mayoría de actividades están repetidas escritas de distintas maneras, por lo tanto intentaré unificarlas

In [20]:
sa_clean.Activity.value_counts().head(60)

Surfing                           971
Swimming                          869
Fishing                           431
Spearfishing                      333
Bathing                           162
Wading                            149
Diving                            127
Standing                           99
Snorkeling                         89
Scuba diving                       76
Body boarding                      61
Body surfing                       49
Swimming                           47
Kayaking                           33
Treading water                     32
Fell overboard                     32
Pearl diving                       32
Boogie boarding                    29
Free diving                        29
Windsurfing                        19
Walking                            17
Boogie Boarding                    16
Shark fishing                      15
Floating                           14
Canoeing                           13
Fishing                            13
Surf-skiing 

In [23]:
def activities(x):
        if x in ["Bathing","Swimming ", "Body surfing", "Free diving", "Free diving ", "Freediving", "Freedom Swimming", "Freedom swimming", "Skin diving"]:
            return "Swimming"
        elif x in ["Surfing", "Surfing (sitting on his board)", "Body boarding", "Boogie Boarding", "Boogie boarding", "Surf skiing", "Surf-skiing", "Sitting on surfboard", "Body-boarding"]:
            return "Surfing"
        elif x in ["Spearfishing", "Shark fishing","Fishing ", "Fishing for sharks"]:
            return "Fishing"
        elif x in ["Scuba diving", "Diving for abalone", "Pearl diving", "Diving", "Sponge diving", "Scuba Diving", "Spearfishing on Scuba", "Diving for trochus", "Free diving for abalone"]:
            return "Snorkeling"
        elif x in ["Standing", "Walking"]:
            return "Wading"
        elif x in ["Treading water", "Floating", "Dangling feet in the water"]:
            return "Treading water"
        elif x in ["Canoeing", "Rowing", "Kayak Fishing", "Kayaking / Fishing"]:
            return "Kayaking"
        elif x in ["Paddle boarding"]:
            return "Paddle boarding"
        elif x in ["Boating","Sailing", "Fishing for mackerel", "Shipwreck", "Fishing on a boat", "Fishing boat"]:
            return "Boating"
        elif x in ["Fell overboard", "Fell into the water"]:
            return "Fell overboard"
        elif x in ["Kite Surfing"]:
            return "Kite Surfing"
        elif x in ["Windsurfing"]:
            return "Windsurfing"
        elif x in ["Surf fishing"]:
            return "Surf fishing"
        elif x in ["Paddle boarding"]:
            return "Paddle boarding"
        else:
            return "Other"
sa_clean["Activities"] = sa_clean.Activity.apply(activities)

sa_clean.Activities.value_counts().head(60)

Other              3883
Surfing            1121
Fishing             372
Swimming            311
Snorkeling          284
Wading              116
Treading water       51
Fell overboard       41
Kayaking             41
Boating              37
Windsurfing          19
Surf fishing         12
Paddle boarding       8
Kite Surfing          6
Name: Activities, dtype: int64

re.sub(r'surf','Surfing',sa_clean)
re.sub(r'fish','Fishing',sa_clean)
re.sub(r'swim','Swimming',sa_clean)
re.sub(r'div','Swimming',sa_clean)

In [23]:
sa_clean.Activity.value_counts().head(40)

Surfing                 971
Swimming                869
Fishing                 431
Spearfishing            333
Bathing                 162
Wading                  149
Diving                  127
Standing                 99
Snorkeling               89
Scuba diving             76
Body boarding            61
Body surfing             49
Swimming                 47
Kayaking                 33
Fell overboard           32
Pearl diving             32
Treading water           32
Free diving              29
Boogie boarding          29
Windsurfing              19
Walking                  17
Boogie Boarding          16
Shark fishing            15
Floating                 14
Fishing                  13
Canoeing                 13
Surf skiing              12
Rowing                   12
Surf-skiing              12
Surf fishing             12
Fishing for sharks       11
Kayak Fishing            11
Scuba Diving             10
Sponge diving            10
Freediving               10
Sailing             

In [ ]:
re.sub('surf', 'Surfing', sa_clean)
re.sub('div', 'Diving', sa_clean)

## País 

In [50]:
sa_clean.Country.value_counts().head(40)

USA                 2229
AUSTRALIA           1338
SOUTH AFRICA         579
PAPUA NEW GUINEA     134
NEW ZEALAND          128
BRAZIL               112
BAHAMAS              109
MEXICO                89
ITALY                 71
FIJI                  62
PHILIPPINES           61
REUNION               60
NEW CALEDONIA         53
CUBA                  46
MOZAMBIQUE            45
SPAIN                 44
INDIA                 40
EGYPT                 38
JAPAN                 34
CROATIA               34
PANAMA                32
SOLOMON ISLANDS       30
IRAN                  29
JAMAICA               27
FRENCH POLYNESIA      25
GREECE                25
HONG KONG             24
INDONESIA             23
ENGLAND               23
PACIFIC OCEAN         17
ATLANTIC OCEAN        17
COSTA RICA            17
BERMUDA               16
TONGA                 15
VIETNAM               15
SRI LANKA             14
VANUATU               14
MARSHALL ISLANDS      13
FRANCE                13
CANADA                12


## Especie

La columna de especies de tiburones es muy ambigua. En algunos casos sólo se registra el tamaño del tiburón, en otros una descripción y en otros es una incógnita. 

In [51]:
sa_clean.Species.value_counts().head(40)

White shark                                           163
Shark involvement prior to death was not confirmed    105
Invalid                                               102
Shark involvement not confirmed                        88
Tiger shark                                            73
Shark involvement prior to death unconfirmed           68
Bull shark                                             52
4' shark                                               40
6' shark                                               40
Questionable incident                                  35
1.8 m [6'] shark                                       35
Questionable                                           34
1.5 m [5'] shark                                       32
1.2 m [4'] shark                                       27
5' shark                                               26
3' shark                                               26
2 m shark                                              25
4' to 5' shark

## Area


In [56]:
sa_clean.Area.value_counts().head(40)

Florida                  1037
New South Wales           486
Queensland                311
Hawaii                    298
California                290
KwaZulu-Natal             213
Western Cape Province     195
Western Australia         189
South Carolina            160
Eastern Cape Province     160
South Australia           104
North Carolina            101
Victoria                   90
Pernambuco                 74
Texas                      73
Torres Strait              70
North Island               67
New Jersey                 52
Tasmania                   41
South Island               40
New York                   30
Oregon                     29
Abaco Islands              23
Northern Territory         23
Central Province           22
Havana Province            19
Virginia                   19
South Province             17
Puerto Rico                16
Massachusetts              16
Gaza                       16
Madang Province            16
Veracruz                   15
Alabama   

In [24]:
sa_clean.Time.value_counts().head(40)



def daytime(x):
        if x :
            
        
        
Day_Time

Afternoon         187
11h00             128
Morning           121
12h00             109
15h00             108
16h00             101
14h00              97
16h30              74
14h30              73
17h00              73
13h00              72
17h30              70
18h00              69
15h30              66
Night              62
11h30              61
13h30              60
10h00              59
09h00              50
10h30              49
12h30              36
Late afternoon     35
09h30              35
Evening            34
07h30              33
18h30              32
08h00              31
08h30              26
19h00              23
07h00              22
17h15              18
13h20              17
15h45              17
14h45              17
16h45              16
11h45              15
Dusk               15
11h15              15
20h00              13
10h45              13
Name: Time, dtype: int64

# EXPORTAR

In [ ]:
df.to_csv(r'Path where you want to store the exported CSV file\File Name.csv')